In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifier,
    SGDClassifier,
    PassiveAggressiveClassifier,
    Perceptron,
)
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.neighbors import (
    KNeighborsClassifier,
    RadiusNeighborsClassifier,
    NearestCentroid,
)
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    BaggingClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    VotingClassifier,
)
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import *
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,
    BaggingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
)
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from tabulate import tabulate  # Import the tabulate library
from sklearn.neural_network import MLPClassifier

In [4]:
!head /content/car.data


vhigh,vhigh,2,2,small,low,unacc
vhigh,vhigh,2,2,small,med,unacc
vhigh,vhigh,2,2,small,high,unacc
vhigh,vhigh,2,2,med,low,unacc
vhigh,vhigh,2,2,med,med,unacc
vhigh,vhigh,2,2,med,high,unacc
vhigh,vhigh,2,2,big,low,unacc
vhigh,vhigh,2,2,big,med,unacc
vhigh,vhigh,2,2,big,high,unacc
vhigh,vhigh,2,4,small,low,unacc


In [5]:
df = pd.read_csv('/content/car.data', header=None)


In [6]:
df

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [8]:
df.isna().sum()


,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0


In [11]:
X = df.drop(6, axis =1)
y = df[6]

In [12]:
X

,0,1,2,3,4,5
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med
...,...,...,...,...,...,...
1723,low,low,5more,more,med,med
1724,low,low,5more,more,med,high
1725,low,low,5more,more,big,low
1726,low,low,5more,more,big,med


In [13]:
y

,6
0,unacc
1,unacc
2,unacc
3,unacc
4,unacc
...,...
1723,good
1724,vgood
1725,unacc
1726,good


In [16]:

# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [29]:
# Encode categorical features in X using one-hot encoding (to handle categorical variables properly)
X_encoded = pd.get_dummies(X, drop_first=True)


In [30]:
# Check for missing values in the features
if X_encoded.isnull().sum().any():
    print("There are missing values in the features. Please clean the data.")
else:
    print("No missing values in the features.")

No missing values in the features.


In [31]:
# Encode the target variable y
le = LabelEncoder()
y_encoded = le.fit_transform(y)


In [32]:
# Display class distribution before balancing
print(f"Class distribution before SMOTE: {Counter(y_encoded)}")

Class distribution before SMOTE: Counter({2: 1210, 0: 384, 1: 69, 3: 65})


In [35]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter

# Assuming df is your DataFrame and it has been loaded already
# Separate features (X) and target (y)
X = df.drop(6, axis=1)  # Features
y = df[6]  # Target (Class)

# Encode categorical features in X using one-hot encoding (to handle categorical variables properly)
X_encoded = pd.get_dummies(X, drop_first=True)

# Check for non-numeric columns and convert them explicitly to numeric values if needed
# Ensure all columns are numeric
X_encoded = X_encoded.apply(pd.to_numeric, errors='coerce')

# Check for any boolean columns and ensure they are treated as numeric (0 or 1)
X_encoded = X_encoded.astype(float)

# Check if there are any NaN or infinite values
if X_encoded.isnull().sum().any() or (X_encoded == float('inf')).sum().any():
    print("There are NaN or infinite values in the features. Please clean the data.")
    # Replace NaN with the median or another strategy of choice
    X_encoded.fillna(X_encoded.median(), inplace=True)

# Encode the target variable y
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Display class distribution before balancing
print(f"Class distribution before SMOTE: {Counter(y_encoded)}")

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_encoded, y_encoded)

# Display class distribution after balancing
print(f"Class distribution after SMOTE: {Counter(y_resampled)}")

# (Optional) Decode the target variable back to original labels
y_resampled_decoded = le.inverse_transform(y_resampled)

# Convert back to a DataFrame (if needed)
X_resampled_df = pd.DataFrame(X_resampled, columns=X_encoded.columns)
y_resampled_df = pd.DataFrame(y_resampled_decoded, columns=['Class'])

# Verify the first few rows of the balanced dataset
print(X_resampled_df.head())
print(y_resampled_df.head())


Class distribution before SMOTE: Counter({2: 1210, 0: 384, 1: 69, 3: 65})
Class distribution after SMOTE: Counter({2: 1210, 0: 1210, 3: 1210, 1: 1210})
   0_low  0_med  0_vhigh  1_low  1_med  1_vhigh  2_3  2_4  2_5more  3_4  \
0    0.0    0.0      1.0    0.0    0.0      1.0  0.0  0.0      0.0  0.0   
1    0.0    0.0      1.0    0.0    0.0      1.0  0.0  0.0      0.0  0.0   
2    0.0    0.0      1.0    0.0    0.0      1.0  0.0  0.0      0.0  0.0   
3    0.0    0.0      1.0    0.0    0.0      1.0  0.0  0.0      0.0  0.0   
4    0.0    0.0      1.0    0.0    0.0      1.0  0.0  0.0      0.0  0.0   

   3_more  4_med  4_small  5_low  5_med  
0     0.0    0.0      1.0    1.0    0.0  
1     0.0    0.0      1.0    0.0    1.0  
2     0.0    0.0      1.0    0.0    0.0  
3     0.0    1.0      0.0    1.0    0.0  
4     0.0    1.0      0.0    0.0    1.0  
   Class
0  unacc
1  unacc
2  unacc
3  unacc
4  unacc


In [36]:
X_resampled_df

,0_low,0_med,0_vhigh,1_low,1_med,1_vhigh,2_3,2_4,2_5more,3_4,3_more,4_med,4_small,5_low,5_med
0,0.0,0.0,1.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,1.0,0.0,0.0,0.316246,0.000000,0.0,0.0,0.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4836,0.0,1.0,0.0,0.000000,1.000000,0.0,0.0,0.545106,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4837,1.0,0.0,0.0,0.019483,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4838,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.772591,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [37]:
y_resampled_df

,Class
0,unacc
1,unacc
2,unacc
3,unacc
4,unacc
...,...
4835,vgood
4836,vgood
4837,vgood
4838,vgood


In [39]:
### It will zero variance features
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(X_resampled_df.select_dtypes(exclude = "object"))


VarianceThreshold(threshold=0)

In [40]:

constant_columns = [column for column in X.select_dtypes(exclude = "object").columns
                    if column not in X.select_dtypes(exclude = "object").columns[var_thres.get_support()]]

print(len(constant_columns))

0


In [41]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [43]:
corr_features = correlation(X_resampled_df, 0.8)
len(set(corr_features))

0

In [44]:
y_resampled_df.value_counts()


,count
Class,
acc,1210
good,1210
unacc,1210
vgood,1210


In [45]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled_df, y_resampled_df, test_size=0.20, random_state = 20)


In [46]:
x_train.shape, x_test.shape


((3872, 15), (968, 15))

In [47]:
num_features = x_train.select_dtypes(exclude = "object")
cat_features = x_train.select_dtypes(include = "object")

In [48]:
num_features


,0_low,0_med,0_vhigh,1_low,1_med,1_vhigh,2_3,2_4,2_5more,3_4,3_more,4_med,4_small,5_low,5_med
1122,0.000000,1.000000,0.0,0.00000,1.00000,0.0,1.0,0.000000,0.000000,1.0,0.0,0.000000,0.0,1.0,0.0
2593,0.347254,0.652746,0.0,1.00000,0.00000,0.0,0.0,1.000000,0.000000,0.0,1.0,1.000000,0.0,0.0,1.0
3167,1.000000,0.000000,0.0,0.00000,1.00000,0.0,0.0,0.606136,0.393864,0.0,1.0,0.000000,1.0,0.0,0.0
2190,0.000000,0.000000,0.0,0.00000,0.00000,0.0,0.0,0.000000,1.000000,1.0,0.0,0.877223,0.0,0.0,1.0
4601,1.000000,0.000000,0.0,1.00000,0.00000,0.0,0.0,0.000000,0.216157,0.0,1.0,0.216157,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607,1.000000,0.000000,0.0,0.00000,1.00000,0.0,0.0,0.000000,1.000000,1.0,0.0,1.000000,0.0,0.0,0.0
3915,0.000000,1.000000,0.0,1.00000,0.00000,0.0,0.0,0.044377,0.955623,0.0,1.0,0.000000,0.0,0.0,0.0
1428,1.000000,0.000000,0.0,0.00000,0.00000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.0,1.0,0.0
4367,0.000000,1.000000,0.0,0.13074,0.86926,0.0,1.0,0.000000,0.000000,0.0,1.0,1.000000,0.0,0.0,0.0


In [49]:
cat_features

""
1122
2593
3167
2190
4601
...
1607
3915
1428
4367


In [52]:
y_train

,Class
1122,unacc
2593,good
3167,good
2190,acc
4601,vgood
...,...
1607,vgood
3915,vgood
1428,unacc
4367,vgood


In [53]:
step1 = ColumnTransformer(
    transformers=[
        ('col_tnf', OneHotEncoder(drop='first', handle_unknown='ignore'),cat_features.columns),
        ('num_tnf', StandardScaler(),num_features.columns )
    ],
    remainder='passthrough'
)

In [54]:
step1

ColumnTransformer(remainder='passthrough',
                  transformers=[('col_tnf',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 Index([], dtype='object')),
                                ('num_tnf', StandardScaler(),
                                 Index(['0_low', '0_med', '0_vhigh', '1_low', '1_med', '1_vhigh', '2_3', '2_4',
       '2_5more', '3_4', '3_more', '4_med', '4_small', '5_low', '5_med'],
      dtype='object'))])

In [55]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier
from tqdm import tqdm
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, LSTM, GRU, Embedding, SimpleRNN, Dropout


# Ignore all warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import *

def evaluate_model(model, x_train, y_train, x_test, y_test):
    # Preprocess 'previous_year_rating' column


    pipe = Pipeline([
        ('Transform', step1),  # You need to define the 'step1' transformer
        ('model', model),
    ])

    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)

    # Use appropriate classification metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred,  average="macro")
    f1 = f1_score(y_test, y_pred,  average="macro")

    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    return accuracy, precision, recall, f1, cm


# Initialize different classification models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest Classifier': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier(),
    'AdaBoost Classifier': AdaBoostClassifier(),
    'Bagging Classifier': BaggingClassifier(),
    'Extra Trees Classifier': ExtraTreesClassifier(),
    'Support Vector Classifier': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree Classifier': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
    'HistGradientBoosting Classifier': HistGradientBoostingClassifier(),
    'SGD Classifier': SGDClassifier(),
    'Passive Aggressive Classifier': PassiveAggressiveClassifier(),
    'Ridge Classifier': RidgeClassifier(),
    'MLP Neural Network': MLPClassifier(hidden_layer_sizes=(10,), max_iter=100),

}
# Create a list to store the results
results = []

# Evaluate and print classification metrics for different models
for model_name, model in tqdm(models.items()):
    accuracy, precision, recall, f1, cm = evaluate_model(model, x_train, y_train, x_test, y_test)
    results.append([model_name, accuracy, precision, recall, f1, cm])

# Print the results in a tabular form
headers = ["Model", "Accuracy", "Precision", "Recall", "F1 Score", "Confusion Matrix"]
print(tabulate(results, headers, tablefmt="grid"))

max_accuracy_index = np.argmax([result[1] for result in results])
best_model_name = results[max_accuracy_index][0]
print(f"\nThe model with the highest accuracy is: {best_model_name} (Accuracy: {results[max_accuracy_index][1]:.4f})")


100%|██████████| 15/15 [00:08<00:00,  1.73it/s]

+---------------------------------+------------+-------------+----------+------------+---------------------+
| Model                           |   Accuracy |   Precision |   Recall |   F1 Score | Confusion Matrix    |
+=================================+============+=============+==========+============+=====================+
| Logistic Regression             |   0.969008 |    0.968968 | 0.969386 |   0.968925 | [[227   7   2   5]  |
|                                 |            |             |          |            |  [  0 225   0   2]  |
|                                 |            |             |          |            |  [ 12   1 238   1]  |
|                                 |            |             |          |            |  [  0   0   0 248]] |
+---------------------------------+------------+-------------+----------+------------+---------------------+
| Random Forest Classifier        |   0.981405 |    0.98164  | 0.981554 |   0.98148  | [[228   2   9   2]  |
|                  

In [57]:
y_train.value_counts()

,count
Class,
good,983
acc,969
vgood,962
unacc,958


In [65]:

model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=100)
model.fit(x_train, y_train)  # Training the model

# Calculate metrics on the training set
train_accuracy = accuracy_score(y_train, model.predict(x_train))
train_precision = precision_score(y_train, model.predict(x_train), average='macro')
train_recall = recall_score(y_train, model.predict(x_train), average='macro')
train_f1 = f1_score(y_train, model.predict(x_train), average='macro')

# Calculate metrics on the test set
test_accuracy = accuracy_score(y_test, model.predict(x_test))
test_precision = precision_score(y_test, model.predict(x_test), average='macro')
test_recall = recall_score(y_test, model.predict(x_test), average='macro')
test_f1 = f1_score(y_test, model.predict(x_test), average='macro')

# Create a DataFrame
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Train Set': [train_accuracy, train_precision, train_recall, train_f1],
    'Test Set': [test_accuracy, test_precision, test_recall, test_f1]
})

metrics_df

,Metric,Train Set,Test Set
0,Accuracy,0.966942,0.958678
1,Precision,0.967971,0.959445
2,Recall,0.966725,0.959393
3,F1 Score,0.966680,0.958759
